# Visualization of Heatmaps

From real data, or simulated data, we here show the different heatmaps that are computed.

In [1]:
import numpy as np
import pandas as pd
import mne
import os
import json
import os.path as op
from pathlib import Path
import collections
from natsort import natsorted

from sklearn.datasets import make_spd_matrix
from mne.io import RawArray
from mne import create_info
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids
import mne
mne.utils.use_log_level('error')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

import dabest

from eztrack.io.base import _add_desc_to_bids_fname, concatenate_derivs
from eztrack.io import read_derivative_npy, DerivativeArray, create_deriv_info
from eztrack.fragility import (lds_raw_fragility, DiscreteLinearSystem, 
                               SystemIDModel, StructuredPerturbationModel)
from eztrack.viz import _load_turbo, generate_heatmap
from eztrack.posthoc.hypo import compute_null

_load_turbo()

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [13]:
root = Path("/Users/adam2392/Dropbox/resection_tvb/")
deriv_root = root / 'derivatives'
figures_path = deriv_root / "figures"

sampling_res = '1000Hz'  # 'originalsampling'
deriv_chain = Path('radius1.25') / 'fragility' / reference
deriv_path = deriv_root / deriv_chain

In [22]:
subjects = ['id008gc', 'id013pg']

subject = 'id008gc'
acquisition = "ecog"
acquisition = 'seeg'
datatype = "ieeg"
extension = ".npy"
task = 'ictal'
run = '01'
session = "extraoperative"  # only one session

# analysis parameters
reference = 'monopolar'
desc = 'perturbmatrix'


In [26]:
bids_path = BIDSPath(subject=subject, session=session,task=task,
                    acquisition=acquisition, run=run, suffix=datatype,
                    datatype=datatype, extension=extension,
                    root=root, check=False)
print(bids_path)

/Users/adam2392/Dropbox/resection_tvb/sub-id008gc/ses-extraoperative/ieeg/sub-id008gc_ses-extraoperative_task-ictal_acq-seeg_run-01_ieeg.npy


In [27]:
deriv_fname = _add_desc_to_bids_fname(bids_path.basename, description=desc)
print(deriv_fname)

sub-id008gc_ses-extraoperative_task-ictal_acq-seeg_run-01_desc-perturbmatrix_ieeg.npy


In [28]:
deriv_fpath = deriv_root / deriv_chain / f'sub-{subject}' / deriv_fname
print(deriv_fpath)
print(deriv_fpath.exists())

/Users/adam2392/Dropbox/resection_tvb/derivatives/radius1.25/fragility/monopolar/sub-id008gc/sub-id008gc_ses-extraoperative_task-ictal_acq-seeg_run-01_desc-perturbmatrix_ieeg.npy
True


# Load Data

## Load Resection Data

# Visualize Data